In [1]:
# import sympy

In [2]:
# demo of sympy
from sympy.core.symbol import symbols
from sympy.solvers.solveset import nonlinsolve
x, y = symbols('x, y', real=True)
nonlinsolve([4*x, 2*x*y - 6*y**2 + 2*y], [x, y])

{(0, 0), (0, 1/3)}

In [3]:
# first im ignoring death rate and combining the two infection rates
s, i, r, d = symbols('s, i, r, d', real=True)
b, N, a = symbols('b, N, a', real=True)
nonlinsolve([(-b*s*i)/N, (b*s*i)/N - r*i - d*i, r*i - a*r, ], [s, i, r, d])

{(0, 0, 0, d), (0, a, r, -r), (0, i, 0, 0), (s, 0, 0, d)}